In [ ]:
import json
import os
import pandas as pd
import numpy as np
import re

current_dir = os.getcwd()
parent_dir = os.path.join(current_dir, os.pardir, os.pardir)
parent_dir = os.path.abspath(parent_dir)

bronze_dir = os.path.join(parent_dir, 'datasets', 'motorlist', '2_bronze')
bronze_files = os.listdir(bronze_dir)
silver_dir = os.path.join(parent_dir, 'datasets', 'motorlist', '3_silver')
silver_files = os.listdir(silver_dir)
motorlist_relationships_dir = os.path.join(parent_dir, 'datasets', 'motorlist', 'relationships')

km77_dir = os.path.join(parent_dir, 'datasets', 'km77')
km77_data_dir = os.path.join(km77_dir, 'Transformed_data')
km77_folders = os.listdir(km77_data_dir)
km77_relationships_dir = os.path.join(km77_dir, 'relationships')

In [ ]:
brand = 'Audi'

### 1. KM 77

In [ ]:
brand_folder = os.path.join(km77_data_dir, brand)
brand_files = os.listdir(brand_folder)

In [ ]:
# Open all brand files, extract name and save in a set
brand_names = set()
for file in brand_files:
    with open(os.path.join(brand_folder, file), 'r', encoding='utf-8') as f:
        data = json.load(f)
        name = data['name']
        brand_names.add(name)

brand_names = sorted(list(brand_names))

In [ ]:
#create a dataframe with the brand data
brand_df = pd.DataFrame(brand_names, columns=['km77_name'])
# Create empty column 'cleaned_name'
brand_df['cleaned_name'] = ''
# Rearrenge the columns so clenaed_name is the first column
brand_df = brand_df[['cleaned_name', 'km77_name']]
brand_df

,cleaned_name,km77_name
0,,A2
1,,A3 3 puertas
2,,A3 5 puertas
3,,A3 Cabrio
4,,A3 Sedan
5,,A3 Sedán
6,,A3 Sportback
7,,A3 allstreet
8,,A4 4 puertas
9,,A4 Avant


In [ ]:
# Save the dataframe to a xsxl file
brand_df.to_excel(os.path.join(km77_relationships_dir, f'{brand.lower()}_cars.xlsx'), index=False)

### 2. Motorlist

In [2]:
# create a dataframe with all the json files
dicts = []

for file in silver_files:
    with open(os.path.join(silver_dir, file), 'r', encoding="utf-8") as f:
        data = json.load(f)
        dicts.append(data)

df = pd.DataFrame(dicts)

df.head()

,Production years,"Displacement, cc",Fuel system,"Power output, hp","Torque output, Nm",Cylinder block,Block head,"Cylinder bore, mm","Piston stroke, mm",Compression ratio,...,Optimal revs,Oil change volume including oil filter,Oil filters,"Cooling, overall volume",Dry weight (base engine),"Dry weight, kg","Combustion chamber working volume, cc","Engine lifespan, thousand km— official information",Max RPM,Type
0,2008-2015,1598.0,Common Rail,120.0,300 – 320,cast iron R4,aluminum 16v,79.5,80.5,16.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2000-2002,2387.0,Common Rail,140.0,305.0,cast iron R5,aluminum 10v,82.0,90.4,18.45,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2002-2010,1970.0,direct injection,165.0,206.0,cast iron R4,aluminum 16v,83.0,91.0,11.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2003-2010,1910.0,Common Rail,150.0,305.0,cast iron R4,aluminum 16v,82.0,90.4,17.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2005-2010,3195.0,direct injection,260.0,322.0,aluminum V6,aluminum 24v,89.0,85.6,11.25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df['car_info'] = df['car_info'].apply(lambda x: np.array(x))
df['car_info']

0       [[Alfa Romeo MiTo I, 2008], [Alfa Romeo MiTo I...
1       [[Alfa Romeo 156, 2000], [Alfa Romeo 156, 2001...
2       [[Alfa Romeo 156, 2002], [Alfa Romeo 156, 2003...
3       [[Alfa Romeo 147, 2005], [Alfa Romeo 147, 2006...
4       [[Alfa Romeo 159, 2005], [Alfa Romeo 159, 2006...
                              ...                        
1359                                                   []
1360                                                   []
1361                                                   []
1362    [[Ford Scorpio, 1991], [Ford Scorpio, 1992], [...
1363    [[Ford Edge, 2014], [Ford F-Series, 2014], [Fo...
Name: car_info, Length: 1364, dtype: object

In [10]:
# remove the empty lists of the car_info column
empty_car_infor_filt = df['car_info'].map(len) > 0
clean_car_df = df[empty_car_infor_filt]
clean_car_df

,Production years,"Displacement, cc",Fuel system,"Power output, hp","Torque output, Nm",Cylinder block,Block head,"Cylinder bore, mm","Piston stroke, mm",Compression ratio,...,Optimal revs,Oil change volume including oil filter,Oil filters,"Cooling, overall volume",Dry weight (base engine),"Dry weight, kg","Combustion chamber working volume, cc","Engine lifespan, thousand km— official information",Max RPM,Type
0,2008-2015,1598.0,Common Rail,120.0,300 – 320,cast iron R4,aluminum 16v,79.5,80.5,16.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2000-2002,2387.0,Common Rail,140.0,305.0,cast iron R5,aluminum 10v,82.0,90.4,18.45,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2002-2010,1970.0,direct injection,165.0,206.0,cast iron R4,aluminum 16v,83.0,91.0,11.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2003-2010,1910.0,Common Rail,150.0,305.0,cast iron R4,aluminum 16v,82.0,90.4,17.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2005-2010,3195.0,direct injection,260.0,322.0,aluminum V6,aluminum 24v,89.0,85.6,11.25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1355,2005-2010,2400.0,Common Rail,185.0,400.0,aluminum R5,aluminum 20v,81.0,93.15,17.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1356,1995-2001,2460.0,direct injection,140.0,290.0,cast iron R5,aluminum 10v,81.0,95.5,20.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1357,NaN,5480.0,NaN,NaN,NaN,NaN,NaN,"98,43",120.0,NaN,...,NaN,NaN,NaN,NaN,NaN,580.0,48.07,NaN,NaN,NaN
1362,NaN,1812.0,carburetor,82.0,135.0,NaN,NaN,80.0,60.14,8.75,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,V-shaped


In [27]:
# Get all the unique values of the clean_car_df and the car_info column considering only the first element inside each list of the array
all_model_cars = clean_car_df['car_info'].apply(lambda x: x[:,0]).explode().unique()
# Transform the array into a list and transform the np.str_ into a string
all_model_cars = [str(x) for x in all_model_cars]
all_model_cars

['Alfa Romeo MiTo I',
 'Fiat Bravo II',
 'Lancia Delta III',
 'Alfa Romeo 156',
 'Alfa Romeo 166',
 'Lancia Lybra',
 'Alfa Romeo GT II',
 'Alfa Romeo GTV II',
 'Alfa Romeo Spider V',
 'Alfa Romeo 147',
 'Fiat Stilo I',
 'Alfa Romeo 159',
 'Alfa Romeo Brera I',
 'Alfa Romeo Spider VI',
 'Alfa Romeo 4C',
 'Alfa Romeo Giulietta',
 'Alfa Romeo 145',
 'Alfa Romeo 146',
 'Alfa Romeo 155',
 'Audi 80 B3',
 'Audi 100 C4',
 'Audi A6 C4',
 'Audi 80 B4',
 'Audi A4 B5',
 'Audi A8 D2',
 'Audi S4',
 'Audi S6',
 'Audi 100 C3',
 'Audi 80',
 'Audi 100',
 'Audi A4',
 'Audi A6',
 'Audi A6 C5',
 'Volkswagen Passat B5',
 'Audi A3 1 (8L)',
 'Seat Cordoba 1 (6K)',
 'Seat Ibiza 2 (6K)',
 'Seat Leon 1 (1M)',
 'Seat Toledo 2 (1M)',
 'Skoda Octavia 1 (1U)',
 'Volkswagen Bora 1 (1J)',
 'Volkswagen Golf 4 (1J)',
 'Volkswagen Polo 3 Classic (6V)',
 'Volkswagen Passat',
 'Audi A6 Allroad',
 'Audi A8',
 'Audi A4 B7',
 'Audi A6 C6',
 'Audi A8 D3',
 'Audi Q7',
 'Volkswagen Touareg',
 'Audi A4 B6 (8E)',
 'Audi A4 B7 (8E)

In [ ]:
# get all the cars that have the brand word in it:
# We put all the cars in lower case and the brand in lower case and check if the brand is in the car name
brand_cars = sorted([x for x in all_model_cars if brand.lower() in x.lower()])
brand_cars

['Audi 100',
 'Audi 100 C3',
 'Audi 100 C4',
 'Audi 80',
 'Audi 80 B2',
 'Audi 80 B3',
 'Audi 80 B4',
 'Audi 90 B3',
 'Audi A1',
 'Audi A2',
 'Audi A3',
 'Audi A3 1',
 'Audi A3 1 (8L)',
 'Audi A3 2',
 'Audi A3 2 (8P)',
 'Audi A4',
 'Audi A4 B5',
 'Audi A4 B6',
 'Audi A4 B6 (8E)',
 'Audi A4 B7',
 'Audi A4 B7 (8E)',
 'Audi A4 B8',
 'Audi A5 1.8 TFSI',
 'Audi A6',
 'Audi A6 Allroad',
 'Audi A6 C4',
 'Audi A6 C5',
 'Audi A6 C5 (4B)',
 'Audi A6 C6',
 'Audi A6 C6 (4F)',
 'Audi A6 C7',
 'Audi A7',
 'Audi A7 C7',
 'Audi A8',
 'Audi A8 D2',
 'Audi A8 D3',
 'Audi A8 D3 (4E)',
 'Audi A8 D4',
 'Audi A8 D5',
 'Audi Q3',
 'Audi Q3 1 (8U)',
 'Audi Q7',
 'Audi Q8 1',
 'Audi RS Q3',
 'Audi RS Q3 2 (F3)',
 'Audi RS3',
 'Audi RS3 3 (8V)',
 'Audi RS4',
 'Audi RS5',
 'Audi RS6 C7',
 'Audi RS7 C7',
 'Audi S4',
 'Audi S6',
 'Audi S8',
 'Audi SQ7',
 'Audi SQ8',
 'Audi TT',
 'Audi TT 2 (8J)',
 'Audi TT RS',
 'Audi TT RS 3 (8S)']

In [ ]:
# Create a dataframe with the list of the cars of the brand
brand_df = pd.DataFrame(brand_cars, columns=['motolist_name'])
# Create three empty columns 'cleaned_name'
brand_df['cleaned_name'] = ''
# Rearrenge the columns so clenaed_name is the first column
brand_df = brand_df[['cleaned_name', 'motolist_name']]
brand_df

,cleaned_name,motolist_name
0,,Audi 100
1,,Audi 100 C3
2,,Audi 100 C4
3,,Audi 80
4,,Audi 80 B2
5,,Audi 80 B3
6,,Audi 80 B4
7,,Audi 90 B3
8,,Audi A1
9,,Audi A2


In [ ]:
# Save the dataframe to a xsxl file
brand_df.to_excel(os.path.join(motorlist_relationships_dir, f'{brand.lower()}_cars.xlsx'), index=False)

### 3. Wallapop